In [2]:
from nuscenes.nuscenes import NuScenes
from pyquaternion.quaternion import Quaternion
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix
from nuscenes.utils.data_classes import PointCloud, LidarPointCloud, RadarPointCloud, Box
import numpy as np
import shutil
from PIL import Image
import os.path as osp
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
from matplotlib import cm
from sphere import Sphere
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from functools import partial
from dh_grid import DHGrid
from sphere import Sphere
from img_sphere import ImageSphere, ColorImageSphere
from metrics import *
from average_meter import AverageMeter
from utils import Utils

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
# export_ds = '/mnt/data/datasets/nuscenes/processed'
export_ds = '/media/scratch/berlukas/nuscenes'
# sem_clouds_filename = f"{export_ds}/sem_clouds3.npy"
# sem_clouds_filename = f"{export_ds}/kitti_processed/sem_clouds_08_new.npy"
sem_clouds_filename = f"{export_ds}/decoded/sem_clouds1_decoded.npy"

print(f"Loading from sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/nuscenes/decoded/sem_clouds1_decoded.npy
Shape of sem clouds is (11230, 7, 200, 200)


In [4]:
n_total_clouds = sem_cloud_features.shape[0]
n_take = 1000

start = n_total_clouds - n_take
end = n_total_clouds

val_features = np.copy(sem_cloud_features[start:end, :, :, :])
print(f"Shape of val clouds is {val_features.shape}")

Shape of val clouds is (1000, 7, 200, 200)


In [5]:
# np.random.shuffle(val_features)

n_subset = 100
subset_feature = val_features[0:n_subset, :, :, :]
print(f"Shape of subset clouds is {subset_feature.shape}")

filename = f"{export_ds}/sem_clouds_decoded.npy"
np.save(filename, subset_feature)
print(f"Wrote subset features to {filename}.")

Shape of subset clouds is (100, 7, 200, 200)
Wrote subset features to /media/scratch/berlukas/nuscenes/sem_clouds_decoded.npy.


In [67]:
print(np.unique(val_features[:,2,:,:]))

[0. 1. 2. 3. 4. 5. 6.]
